In [1]:
import numpy as np
import pandas as pd
import pickle

## load string db and get combined score ppi network

In [2]:
local_stringdb = 'D:/study/thesis/project/HBDM-main/nn_data/stringdb/'
# load local STRING database and names
df = pd.read_csv(local_stringdb+'9606.protein.info.v12.0.txt', sep='\t', header=0, usecols=['#string_protein_id', 'preferred_name'])
df['preferred_name'] = df['preferred_name'].str.upper()
stringId2name = df.set_index('#string_protein_id')['preferred_name'].to_dict()
name2stringId = df.set_index('preferred_name')['#string_protein_id'].to_dict()
df = pd.read_csv(local_stringdb+'9606.protein.aliases.v12.0.txt', sep='\t', header=0, usecols=['#string_protein_id', 'alias']).drop_duplicates(['alias'], keep='first')
df['alias'] = df['alias'].str.upper()
aliases2stringId = df.set_index('alias')['#string_protein_id'].to_dict()

#string_score_transform = lambda x: -np.log(x/1000)

graph_df = pd.read_csv(local_stringdb+'9606.protein.physical.links.detailed.v12.0.txt', sep=' ', header=0).convert_dtypes().replace(0, float('nan'))
#network['combined_score'] = network['combined_score'].apply(string_score_transform)
graph_df = graph_df[['protein1', 'protein2','combined_score']]
graph_df['protein1'] = graph_df['protein1'].str.replace('9606.ENSP', '')
graph_df['protein2'] = graph_df['protein2'].str.replace('9606.ENSP', '')

In [6]:


file_path = r'D:\study\thesis\project\HBDM-main\datasets\ppi\Aliases2stringId.pkl'

# Serialize and save the Tensor to the file
with open(file_path, 'wb') as file:
    pickle.dump(aliases2stringId, file)
# Close the file
file.close()

In [3]:
graph_df.head(3)

,protein1,protein2,combined_score
0,00000000233,00000257770,311
1,00000000233,00000226004,161
2,00000000233,00000434442,499


In [4]:
len(graph_df)

1477610

## filter with a threshold

In [5]:
threshold = 100
graph_df = graph_df[graph_df['combined_score'] >= threshold]
len(graph_df)

1477610

In [14]:
uniquepair = set()
nodes = set()
sub_graph_df = graph_df[['protein1', 'protein2']]
sub_graph_array = np.array(sub_graph_df.to_numpy(), dtype=int)
for subarray in sub_graph_array:
    result_string = '-'.join(map(str, set(subarray)))
    uniquepair.add(result_string)
    nodes.update(set(subarray))
print(len(uniquepair))
print(len(nodes))

result_array = np.sort(np.array([list(map(int, item.split('-'))) for item in uniquepair]))
value_to_index_mapping = {value: index for index, value in enumerate(sorted(list(nodes)))}
mapped_arr = np.vectorize(value_to_index_mapping.get)(result_array)

output = mapped_arr.T
print(output.shape)

805326
18767
(2, 805326)


In [16]:
# stringint2name = {int(key.split('.')[-1][-11:]): value for key, value in stringId2name.items()}
# index2name = {value: stringint2name[key] for key, value in value_to_index_mapping.items()}
# len(index2name)

In [17]:
# id2index = {value: key for key, value in value_to_index_mapping.items()}

## save results

In [15]:
savepath = 'D:/study/thesis/project/HBDM-main/datasets/ppi/'
np.savetxt(savepath+'sparse_i.txt', output[0], delimiter='\n')
np.savetxt(savepath+'sparse_j.txt', output[1], delimiter='\n')

In [16]:
file_path = savepath+'ppi_index.pkl'

# Serialize and save the Tensor to the file
with open(file_path, 'wb') as file:
    pickle.dump(value_to_index_mapping, file)
# Close the file
file.close()

In [17]:
len(set(graph_df['protein1'].tolist()))

18767

In [21]:
# if the head half is the deduplicated
split_index = len(graph_array) // 2
test = graph_array[:split_index]
if np.array_equal(test, unique_elements):
    print('The arrays are equal.')
else:
    print('The arrays are not equal.')

NameError: name 'graph_array' is not defined

In [ ]:
file_path = savepath+'ppi_name2stringId.pkl'

# Serialize and save the Tensor to the file
with open(file_path, 'wb') as file:
    pickle.dump(name2stringId, file)
# Close the file
file.close()

file_path = savepath+'ppi_aliases2stringId.pkl'

# Serialize and save the Tensor to the file
with open(file_path, 'wb') as file:
    pickle.dump(aliases2stringId, file)
# Close the file
file.close()